In [1]:
import pandas as pd
import string
import re
import numpy as np

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec, KeyedVectors

In [4]:
questions = pd.read_csv("./data/labelled_clean_questions.csv")

Custom Word2Vec + Random Forest Model

In [5]:
sentences = questions["clean_tokenized"].values.astype(str)
questions["Custom Word2Vec + Random Forest Modellean_tokenized"] = questions["clean_tokenized"].apply(lambda x: print(x) if type(x) is float else x.split(" "))
sentences = questions["clean_tokenized"].values
sent = [x for x in sentences if x]
model = Word2Vec(sent, min_count=1)
print(model)

nan
nan
nan
nan
nan
nan
nan
nan
Word2Vec(vocab=197642, size=100, alpha=0.025)


In [6]:
model.wv.save_word2vec_format('model.bin', binary=True)

Create Vectors from Quora Dataset

In [21]:
def avg_vector(arr): 
    arr = np.array([model[x] for x in arr if x in model])
    return np.mean(arr, axis=0)

In [23]:
questions["word2vec"] = questions["tokenized"].apply(avg_vector)

/Users/skadiyala/.local/share/virtualenvs/nlp-tSIpt5ka/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/Users/skadiyala/.local/share/virtualenvs/nlp-tSIpt5ka/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [27]:
train_balanced = questions[questions['target'] == 1]
train_balanced = train_balanced.append(questions[questions['target'] == 0].sample(n = len(train_balanced))).reset_index(drop = True)
train_balanced['target'].value_counts()

balanced_text = train_balanced["word2vec"]

train_text, test_text, target_train, target_test = train_test_split(balanced_text, train_balanced["target"], stratify=train_balanced["target"], test_size=0.25)


In [77]:
train_balanced.to_csv("./data/questions_clean_word2vec.csv")

In [47]:
arr = list(train_text)

In [68]:
num_features = 60

def same_num_features(train_text):
    new = []
    padding = [0]*60
    for x in train_text: 
        x = x.tolist()
        x.append(padding)
        new.append(x[0:60])
    return new

train_text = same_num_features(train_text)

In [72]:
test_text = same_num_features(test_text)

In [70]:
clf = RandomForestClassifier(max_depth= 10, min_samples_leaf= 3,
                         min_samples_split= 3, n_estimators= 800)
clf.fit(train_text, target_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=3, min_samples_split=3,
                       n_estimators=800)

In [75]:
!pip install pickle-mixin
import pickle
with open('word2vec_random_forest.pkl','wb') as f:
    pickle.dump(clf,f)

  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=6008 sha256=26b5fa0c345d77c197907e42cc712f1a77f540ce4143fbc00e1c9079314ddf64
  Stored in directory: /Users/skadiyala/Library/Caches/pip/wheels/d0/70/0b/673e09a7ed429660d22352a1b117b4f616a8fc054bdd7eb157
Successfully built pickle-mixin


In [73]:
preds = clf.predict(test_text)
print(classification_report(target_test, preds))

              precision    recall  f1-score   support

           0       0.66      0.69      0.68     20202
           1       0.68      0.65      0.66     20203

    accuracy                           0.67     40405
   macro avg       0.67      0.67      0.67     40405
weighted avg       0.67      0.67      0.67     40405



In [34]:
questions

,qid,question_text,target,tokenized,length,clean_tokenized,word2vec
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0,"['How', 'did', 'Quebec', 'nationalists', 'see'...",14,"[how, did, quebec, nationalist, see, their, pr...","[-0.3253699, -0.21867748, 0.17936249, 1.040233..."
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0,"['Do', 'you', 'have', 'an', 'adopted', 'dog', ...",18,"[do, you, have, an, adopt, dog, how, would, yo...","[-0.43439114, -0.57571846, 0.3039751, 1.102035..."
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0,"['Why', 'does', 'velocity', 'affect', 'time', ...",12,"[whi, doe, veloc, affect, time, ?, doe, veloc,...","[-0.3108829, -0.5737844, 0.45734033, 0.9274742..."
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0,"['How', 'did', 'Otto', 'von', 'Guericke', 'use...",10,"[how, did, otto, von, guerick, use, the, magde...","[-0.53613114, -0.3556412, 0.37920538, 0.949985..."
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0,"['Can', 'I', 'convert', 'montra', 'helicon', '...",16,"[can, i, convert, montra, helicon, d, to, a, m...","[-0.42371547, -0.32201287, 0.27109084, 1.04583..."
...,...,...,...,...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0,"['What', 'other', 'technical', 'skills', 'do',...",18,"[what, other, technic, skill, do, you, need, a...","[-0.43368655, -0.44589114, 0.36363015, 1.05633..."
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0,"['Does', 'MS', 'in', 'ECE', 'have', 'good', 'j...",20,"[doe, ms, in, ece, have, good, job, prospect, ...","[-0.44217533, -0.5285403, 0.4141498, 1.0824167..."
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0,"['Is', 'foam', 'insulation', 'toxic', '?']",5,"[is, foam, insul, toxic, ?]","[-0.28936014, 0.04574925, -0.1001611, 0.978329..."
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0,"['How', 'can', 'one', 'start', 'a', 'research'...",14,"[how, can, one, start, a, research, project, b...","[-0.32839063, -0.6131454, 0.36521408, 1.177455..."
